# Michael Kissiedu

# Project 3: Learn data merging, group comparison, and basic regression analysis.

# Instructor: Dr. Jin

# Teaching Assistant: Ziqiao

# Deadline: Friday March 12th 2021

The goal of this project is to learn how to merge data, perform group comparison, and perform basic regression analysis.

The order of doing the project.

*1. Modules were imported in the beginning.

*2. The long sample created in project 1 was imported

*3. Data cleaning was continuously performed during the project.

*4. Core place records was imported.

*5. Different codes were ran in order to answer the questions relating to the project.

*6. Finally, in addition to the codes, the questions were inserted above the codes which were used to answer the question. Then 

*below the last code relating to the question, an answer was provided.

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import sklearn.linear_model as sl
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import math
from scipy.stats import ttest_ind
from linearmodels import PanelOLS


import scipy
print(scipy.__version__)

1.6.1


The project used three datasets: the  “long sample” created in Project 1, a cross dectional data from Safegrpah called 'core 

place records,' and the 2017 Census definition of 6-digit NAICS codes. The long sample was imported in line 2. "Core place 

records" was imported in line 4. The 2017 Census definition of 6-digit NAICS codes was used to pick businesses used for 

comparison. Every observation in the "core place records" dataset "is a business place, with a unique Safegraph id, 

industry code (6-digit NAICS), brand affiliation, etc."

In [2]:
wlong = pd.read_csv('wlong_raw.csv')
wlong.head()

,id,Rank,date_range_start,date_range_end,visits_by_day,region,raw_visitor_counts,raw_visit_counts,safegraph_place_id,daily_visits,dayname,date,positive_visits,weekend
0,466,1,2020-03-09,2020-03-16,"11,5,1,2,1,7,7",WY,19.0,34.0,sg:0e5816c2c3074793ae5fc6f9c3e6bb7e,11.0,Mon,2020-03-09,1,0
1,552,1,2020-03-09,2020-03-16,"13,13,13,5,13,2,11",WY,41.0,70.0,sg:111ac71acaee4d10b00ef01388ae3e98,13.0,Mon,2020-03-09,1,0
2,1399,1,2020-03-09,2020-03-16,"10,4,4,6,4,0,0",WY,13.0,28.0,sg:2a17146a35e149dabce27332a52f8d1f,10.0,Mon,2020-03-09,1,0
3,2694,1,2020-03-09,2020-03-16,"5,8,5,4,14,8,5",WY,44.0,49.0,sg:5380bbb5685d4e6eb8427f4b1d418772,5.0,Mon,2020-03-09,1,0
4,2752,1,2020-03-09,2020-03-16,"0,1,0,0,2,0,0",WY,2.0,3.0,sg:55dbc9ae0c514561b30d27f7b7d74710,0.0,Mon,2020-03-09,0,0


In [3]:
# Data cleaning is performed with the code below.
wlong['date'] = pd.to_datetime(wlong['date'])

# Question 1a ask  how many observations are in “StateCorePlaces” data and   

# how many unique cities are there in your state?

In [4]:
# Coreplace records were imported using the code below.
filename = 'core_poi-part{}.csv'
places = pd.concat([pd.read_csv(filename.format(i), 
                                dtype = {'naics_code': 'object',
                                         'postal_code': 'object',
                                         'phone_number': 'object'}) 
                    for i in range(1, 6)], ignore_index = True)
places.head()

,safegraph_place_id,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,iso_country_code,phone_number,open_hours,category_tags
0,sg:001341fe7e794ab6bd65bb80759a1ac6,NaN,Old Firehouse School,NaN,NaN,Child Day Care Services,Child Day Care Services,624410,37.890704,-122.118315,984 Moraga Rd,Lafayette,CA,94549,US,+19252844321,"{ ""Mon"": [[""8:00"", ""17:30""]], ""Tue"": [[""8:00"",...",NaN
1,sg:002921847f104c93a6df47bb831dedd2,NaN,Tenth Avenue Park,NaN,NaN,"Museums, Historical Sites, and Similar Institu...",Nature Parks and Other Similar Institutions,712190,42.255103,-89.085241,825 10th St,Rockford,IL,61104,US,NaN,NaN,NaN
2,sg:002d2e9f047d47ca8661abb8939096cf,NaN,Hunter Village Wines & Liquors,NaN,NaN,"Beer, Wine, and Liquor Stores","Beer, Wine, and Liquor Stores",445310,42.214633,-74.220827,7261 Rt 23a,Hunter,NY,12442,US,+15182634184,"{ ""Mon"": [], ""Tue"": [[""13:00"", ""18:00""]], ""Wed...",NaN
3,sg:0060616e5f184585a026a0b7e8192fbd,NaN,Fat Moe's,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511,40.948824,-74.154553,204 Diamond Bridge Ave,Hawthorne,NJ,07506,US,+19739494905,NaN,"Mediterranean Food,Smoothie & Juice Bar,Sandwi..."
4,sg:0061c1d8e124406cbcc97e40eb1dd585,NaN,Jungle Rapids Family Fun Park,NaN,NaN,Amusement Parks and Arcades,Amusement and Theme Parks,713110,34.207249,-77.863010,5320 Oleander Dr,Wilmington,NC,28403,US,+19107910666,NaN,NaN


In [5]:
# Columns not needed were dropped to reduce data size.
places.drop(columns = ['parent_safegraph_place_id', 'safegraph_brand_ids', 'brands', 'street_address', 'phone_number', 'open_hours', 
                       'category_tags', 'iso_country_code'], 
            inplace = True)
places.head()

,safegraph_place_id,location_name,top_category,sub_category,naics_code,latitude,longitude,city,region,postal_code
0,sg:001341fe7e794ab6bd65bb80759a1ac6,Old Firehouse School,Child Day Care Services,Child Day Care Services,624410,37.890704,-122.118315,Lafayette,CA,94549
1,sg:002921847f104c93a6df47bb831dedd2,Tenth Avenue Park,"Museums, Historical Sites, and Similar Institu...",Nature Parks and Other Similar Institutions,712190,42.255103,-89.085241,Rockford,IL,61104
2,sg:002d2e9f047d47ca8661abb8939096cf,Hunter Village Wines & Liquors,"Beer, Wine, and Liquor Stores","Beer, Wine, and Liquor Stores",445310,42.214633,-74.220827,Hunter,NY,12442
3,sg:0060616e5f184585a026a0b7e8192fbd,Fat Moe's,Restaurants and Other Eating Places,Full-Service Restaurants,722511,40.948824,-74.154553,Hawthorne,NJ,07506
4,sg:0061c1d8e124406cbcc97e40eb1dd585,Jungle Rapids Family Fun Park,Amusement Parks and Arcades,Amusement and Theme Parks,713110,34.207249,-77.863010,Wilmington,NC,28403


In [6]:
# Businesses to be compared later are chosen here. Dining and hotels only are kept in the data.
places = places[places['naics_code'].isin(['722511', '722513', '722514', 
                                           '721110', '721120', '721191'])].copy()
places.head()

,safegraph_place_id,location_name,top_category,sub_category,naics_code,latitude,longitude,city,region,postal_code
3,sg:0060616e5f184585a026a0b7e8192fbd,Fat Moe's,Restaurants and Other Eating Places,Full-Service Restaurants,722511,40.948824,-74.154553,Hawthorne,NJ,07506
10,sg:012d0c0a0426477fa6b454e50c9532f2,Holiday Inn Express,Traveler Accommodation,Hotels (except Casino Hotels) and Motels,721110,41.647719,-86.000766,Elkhart,IN,46517
28,sg:021c38c9343a48b4a64a28c2f051dc1d,Bill Gray's,Restaurants and Other Eating Places,Full-Service Restaurants,722511,43.133103,-77.491124,Rochester,NY,14625
32,sg:028a35b9e6594d5d85a8deec90554e23,Wung Thai Restaurant And Sushi,Restaurants and Other Eating Places,Full-Service Restaurants,722511,27.916708,-82.709770,Clearwater,FL,33760
36,sg:029ede0160ce42e0a17f3f5f1b7a6c1e,Golden Wok,Restaurants and Other Eating Places,Full-Service Restaurants,722511,39.652308,-104.810433,Aurora,CO,80014


In [7]:
# A variable “Bizgroup” equal to 0 if a place’s NAICS code falls in the dining group.
# A variable equal to 1 if it falls in the hotel group.
places['Bizgroup'] = np.where(places['naics_code'].isin(['722511', '722513', '722514']), 1, 0)
places.groupby('Bizgroup').size()

Bizgroup
0     68779
1    622299
dtype: int64

In [8]:
# Wyoming is focused on here. This will decrease the number of businesses in Bizgroup (check line 12).
places = places[places['region'] == 'WY']

In [9]:
places.head()

,safegraph_place_id,location_name,top_category,sub_category,naics_code,latitude,longitude,city,region,postal_code,Bizgroup
3811,sg:1f81bb35e9d24f0ab2cabdf8d2c7fcda,Pahaska Tepee Resort,Traveler Accommodation,Hotels (except Casino Hotels) and Motels,721110,44.502499,-109.963807,Cody,WY,82414,0
5562,sg:a0c02e0e5b0a49038ff9617ac5d11ff3,Old Faithful Snow Lodge Geyser Grill,Traveler Accommodation,Hotels (except Casino Hotels) and Motels,721110,44.555071,-110.395743,Yellowstone National Park,WY,82190,0
9039,sg:9d80429296d44fc0b570ba35884e3574,Old Chicago Pizza and Taproom,Restaurants and Other Eating Places,Full-Service Restaurants,722511,41.159139,-104.798990,Cheyenne,WY,82009,1
15202,sg:6b6cb85b44654eb4b1823a356a45b5a9,Village Inn,Restaurants and Other Eating Places,Full-Service Restaurants,722511,42.847213,-106.322012,Casper,WY,82601,1
17973,sg:34e596f1c02f4f31b4a624e4f5083664,Union Bar & Lounge,Restaurants and Other Eating Places,Full-Service Restaurants,722511,42.904650,-108.583369,Hudson,WY,82515,1


In [10]:
len(places)

1734

In [11]:
places['city'].nunique()

126

The answers to question 1a, which ask how many observations do you have in “StateCorePlaces” and how many unique cities are there in your state are stated below.

1,734 observations are in Wyoming state core places records. 126 unique cities are Wyoming state core places records.

Question 1B only requires choosing two groups of businesses to compare. Then ask to state what each group represents, why you are interested in this comparison, and how you expect DailyVisits to differ between these two groups in the four weeks you have studied in Project 1. The answer is below.

Relating to question 1b, dining services and hotels businesses are of interest. I am interested in the two since people need 

food and homes to survive. Therefore, there should be a big amount of data provided by them. The NAICS codes corresponding to 

restaurants, cafeteria, etc., are 722511, 722513, 722514. The NAICS codes for hotel are 721110, 721120, 721191.

A variable `Bizgroup` that identifies my places of interest, with `Bizgroup` = 1 being dining places, and `Bizgroup` = 0 being 

hotels is created. I expect daily visits to be more in the dining service industry compared to the hotel industry.

Question 1c (2 point): Back to python notebook. In the “StateCorePlaces” data you have created in Question 1a, define a variable “bizgroup” equal to 0 if a place’s NAICS code falls in the first group you have chosen in Question 1b, and equal to 1 if it falls in the second group. “bizgroup” should be a missing value for places that do not belong to either group.

 

1. Statewide, how many places have bizgroup=0? how many places have bizgroup=1?

2. Find the top ten cities in your state that have the highest count of unique places (across all NAICS codes). 

3. For these ten cities, count # of places by bizgroup=0 and bizgroup=1 in every city. You can use a table to present this result, where each row is a city, one column shows the count of places with bizgroup=0, and another column shows the count of places with bizgroup=1.

In [12]:
places['Bizgroup'] = np.where(places['naics_code'].isin(['722511', '722513', '722514']), 1, 0)
places.groupby('Bizgroup').size()

Bizgroup
0     506
1    1228
dtype: int64

The answer to part 1 of queston 1c 506 places are hotel businesses. 1,228 places are dining businesses.

In [13]:
dv_sum = (places.groupby('city').agg({'safegraph_place_id': 'count'})
          .sort_values(by = 'safegraph_place_id', ascending = False))
top10 = list(dv_sum.index[:10])
places10 = places.loc[places['city'].isin(top10)]
places10['city'].unique()

array(['Cody', 'Cheyenne', 'Casper', 'Gillette', 'Rawlins', 'Laramie',
       'Jackson', 'Evanston', 'Rock Springs', 'Sheridan'], dtype=object)

The answer to the 2nd part of question 1c is listed above. The top ten cities are cody, Cheyenne, Casper, Gillette, Rawlins, 

laramie, Jackson, Evanston, Rock Spring, and Sheridan.

In [14]:
top_biz = places10[['city', 'Bizgroup', 'safegraph_place_id']].groupby(['city', 'Bizgroup']).agg({'safegraph_place_id': 'size'}).unstack()

In [15]:
top_biz

safegraph_place_id     
Bizgroup                      0    1
city                                
Casper                       23  129
Cheyenne                     32  153
Cody                         37   53
Evanston                     12   34
Gillette                     26   71
Jackson                      41   61
Laramie                      24   69
Rawlins                      21   30
Rock Springs                 17   61
Sheridan                     21   53

The answer to the 3rd part of question 1c continuess here. The table above shows the number of businesses of each category in each city.

# Part 2: Merge Daily Visits Data with StateCorePlaces.

# Question 2a stated to merge "StateCorePlaces" with the “long” sample used in Project 1, by Safegraph place id.

Then answer, how many observations are matched? How many observations appear in StateCorePlaces but not in the “long” sample? How many observations appear in the “long” sample but not in “StateCorePlaces”?

In [16]:
# merge data
df = pd.merge(wlong, places, on = 'safegraph_place_id', how = 'inner', suffixes=('', '_y'))

# drop duplicated columns (column names end with '_y')
df.drop(columns = df.filter(regex='_y$').columns.values, inplace = True)
df.head()

,id,Rank,date_range_start,date_range_end,visits_by_day,region,raw_visitor_counts,raw_visit_counts,safegraph_place_id,daily_visits,dayname,date,positive_visits,weekend,location_name,top_category,sub_category,naics_code,latitude,longitude,city,postal_code,Bizgroup
0,2694,1,2020-03-09,2020-03-16,"5,8,5,4,14,8,5",WY,44.0,49.0,sg:5380bbb5685d4e6eb8427f4b1d418772,5.0,Mon,2020-03-09,1,0,Tacos Mexico,Restaurants and Other Eating Places,Full-Service Restaurants,722511,42.83767,-106.300181,Casper,82601,1
1,3859526,1,2020-03-16,2020-03-23,"10,3,6,5,10,0,1",WY,25.0,35.0,sg:5380bbb5685d4e6eb8427f4b1d418772,10.0,Mon,2020-03-16,1,0,Tacos Mexico,Restaurants and Other Eating Places,Full-Service Restaurants,722511,42.83767,-106.300181,Casper,82601,1
2,7601965,1,2020-03-23,2020-03-30,"6,3,1,4,5,2,0",WY,19.0,21.0,sg:5380bbb5685d4e6eb8427f4b1d418772,6.0,Mon,2020-03-23,1,0,Tacos Mexico,Restaurants and Other Eating Places,Full-Service Restaurants,722511,42.83767,-106.300181,Casper,82601,1
3,11239563,1,2020-03-30,2020-04-06,"0,0,2,4,7,1,1",WY,13.0,15.0,sg:5380bbb5685d4e6eb8427f4b1d418772,0.0,Mon,2020-03-30,0,0,Tacos Mexico,Restaurants and Other Eating Places,Full-Service Restaurants,722511,42.83767,-106.300181,Casper,82601,1
4,2694,2,2020-03-09,2020-03-16,"5,8,5,4,14,8,5",WY,44.0,49.0,sg:5380bbb5685d4e6eb8427f4b1d418772,8.0,Tue,2020-03-10,1,0,Tacos Mexico,Restaurants and Other Eating Places,Full-Service Restaurants,722511,42.83767,-106.300181,Casper,82601,1


In [17]:
a = len(wlong[~wlong['safegraph_place_id'].isin(places['safegraph_place_id'])])

In [18]:
b = len(places[~places['safegraph_place_id'].isin(wlong['safegraph_place_id'])])

In [19]:
print('length of original long data: ', len(wlong))
print('length of original core place data: ', len(places))
print('length of merged data: ', len(df))
print('obs in wlong but not in core place: ', a)
print('obs in core place but not in long: ', b)

length of original long data:  240296
length of original core place data:  1734
length of merged data:  39676
obs in wlong but not in core place:  200620
obs in core place but not in long:  210


In [20]:
# Data was sorted with the code below.
df = df.sort_values(['date', 'id'], ascending = True)

In [21]:
# Order of colums were rearranged with the code below.

first_cols = ['date', 'safegraph_place_id', 'naics_code']
other_cols = [col for col in df.columns if col not in first_cols]
df = df[first_cols + other_cols]
df.head()

,date,safegraph_place_id,naics_code,id,Rank,date_range_start,date_range_end,visits_by_day,region,raw_visitor_counts,raw_visit_counts,daily_visits,dayname,positive_visits,weekend,location_name,top_category,sub_category,latitude,longitude,city,postal_code,Bizgroup
0,2020-03-09,sg:5380bbb5685d4e6eb8427f4b1d418772,722511,2694,1,2020-03-09,2020-03-16,"5,8,5,4,14,8,5",WY,44.0,49.0,5.0,Mon,1,0,Tacos Mexico,Restaurants and Other Eating Places,Full-Service Restaurants,42.837670,-106.300181,Casper,82601,1
28,2020-03-09,sg:667f2869c067452299a7b4d31b97522f,722511,3258,1,2020-03-09,2020-03-16,"14,7,2,24,22,12,12",WY,78.0,93.0,14.0,Mon,1,0,Corona Village,Restaurants and Other Eating Places,Full-Service Restaurants,41.299358,-105.590968,Laramie,82070,1
56,2020-03-09,sg:908713aab1e4471395e7a7fe9acb38ef,722511,19588,1,2020-03-09,2020-03-16,"0,0,0,0,1,0,0",WY,1.0,1.0,0.0,Mon,0,0,China King Buffet,Restaurants and Other Eating Places,Full-Service Restaurants,44.281021,-105.491578,Gillette,82716,1
77,2020-03-09,sg:aac9d2d8d42841e5b2c5db87aba3252d,722511,20309,1,2020-03-09,2020-03-16,"0,0,1,5,0,8,1",WY,11.0,15.0,0.0,Mon,0,0,Spanky's Bar,Restaurants and Other Eating Places,Full-Service Restaurants,41.270939,-110.955981,Evanston,82930,1
98,2020-03-09,sg:f572a3392d524dc1ac67813208bfba16,721110,30133,1,2020-03-09,2020-03-16,"8,5,9,11,9,15,13",WY,17.0,70.0,8.0,Mon,1,0,Wyoming Inn of Jackson Hole,Traveler Accommodation,Hotels (except Casino Hotels) and Motels,43.473637,-110.783691,Jackson,83001,0


The answers to 2a are, 39,676 observations are matched. 210 observations appear in StateCorePlaces but not in the “long” sample. 200,620 observations appear in the “long” sample but not in “StateCorePlaces.”

# Question 2b states, keep the matched records from the merge in Question 2a; keep only the places that have either bizgroup=0 or bizgroup=1 in your top ten cities. Based on this “final” data, present a table that shows the number of observations on each date.

In [22]:
df = pd.merge(wlong, places, on = 'safegraph_place_id', how = 'inner', suffixes = ('', '_y'))
df.drop(columns = df.filter(regex='_y$').columns.values, inplace = True)

In [23]:
df = df.loc[df['city'].isin(top10)].copy()

In [24]:
date = df[['date', 'safegraph_place_id', 'daily_visits']].groupby(['date']).agg({'safegraph_place_id':'count'})
date

,safegraph_place_id
date,
2020-03-09,865
2020-03-10,865
2020-03-11,865
2020-03-12,865
2020-03-13,865
2020-03-14,865
2020-03-15,865
2020-03-16,857
2020-03-17,857


The table in line 23 above answers 2b (present a table that shows the number of observations on each date).

# Independent samples t-test

Question 3a (2 points): Focus on the “final” data. For each date, calculate average dailyvisits for bizgroup=0 and bizgroup=1.  Show the results in a table.  How does the average dailyvisits differ between the two groups? (Write at least one sentence about your takeaway from the table.)

In [25]:
df.groupby('Bizgroup').agg({'daily_visits': 'mean'})

,daily_visits
Bizgroup,
0,4.590151
1,5.310711


Daily visits differs between the two groups by 1. Dinners have 1 more daily visit than hotel. Dinners daily visits is on average higher than hotels.

Question 3b

The t test (also called Student’s T Test) compares two averages (means) and tells you if they are different from each other. The t test also tells you how significant the differences are; In other words it lets you know if those differences could have happened by chance.

*Null hypothesis:* there is no difference between the mean of the two groups.

*Alternative hypothesis:* the mean between the two groups are different.


$$t = \frac{\bar{\mu_a} - \bar{\mu_b}}{\sqrt{\frac{S_a^2}{n_a} + \frac{S_b^2}{n_b}}}$$

$$S^2 = \frac{\sum(a-\mu)^2}{n-1}$$

In [26]:
a = df.loc[df['Bizgroup'] == 1]['daily_visits']
b = df.loc[df['Bizgroup'] == 0]['daily_visits']

t, p = scipy.stats.ttest_ind(a, b, equal_var=False, alternative = 'two-sided')
print('The t-test results are: t-statistics = {}, p-value = {}'.format(t,p))

The t-test results are: t-statistics = 7.16770170369558, p-value = 7.98116343375071e-13


Answer to question 3b: The independent samples t-test has a p-value of 0.000000000000798, which means that the probability of seeing a test statistics as extreme as in this projects case, assuming null hypothesis is true, is very small. Therefore, reject the null on a 95% CI and conclude that there is a difference between the mean dailyvisits between restaurants and other businesses.

To find the 95% Confidence Interval for the difference in means, we can use another package.

In [27]:
#import statsmodels.stats.api as sms
compare = sms.CompareMeans(sms.DescrStatsW(a), sms.DescrStatsW(b))
print(compare.tconfint_diff(usevar='unequal'))

(0.5235117239211909, 0.91760902426361)


In [28]:
# calculate CI by hand
# import math
var_a = a.var(ddof = 1)
var_b = b.var(ddof = 1)
S = math.sqrt(var_a/len(a) + var_b/len(b))
ub = (a.mean() - b.mean()) + 1.96 * S
lb = (a.mean() - b.mean()) - 1.96 * S
print('95% CI for difference in mean is ({:.2f}, {:.2f}).'.format(lb, ub))

95% CI for difference in mean is (0.52, 0.92).


The confidence interval does not include zero in it. A conlusion can be reach that the null is rejected. Yes this confirms my prior in the answers to question 1b.

# part 4: Basic Regrssion analysis

# Question 4a: focus on the “final” data. Define a dummy “Post0313” equal to one if date is at or after March 13, 2020. Define “bizgroup_post” as bizgroup times Post0313. Regress ln(dailyvisits+1) on bizgroup, Post0313, bizgroup_post, and weekend.

# Comment on the estimated coefficient for each of the four variables: Are they statistically significant from zero with 95% confidence? What is the economic meaning of each coefficient? Do you accept or reject the null hypothesis “H0: The national emergency announced on March 13 does not make a different on average dailyvisits to the two groups of businesses”?

In [29]:
# r_date = pd.to_datetime('2020-03-13', infer_datetime_format = True)
#  The dummy variable is defined with the code be df['post0313'].
df['post0313'] = np.where(df['date'] > pd.to_datetime('2020-03-13'), 1, 0)

In [30]:
df['Bizgroup_post'] = df['Bizgroup'] * df['post0313']

In [31]:
results = smf.ols('daily_visits ~ C(Bizgroup) * C(post0313) + C(Bizgroup_post) + C(city) + C(weekend)', data=df).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           daily_visits   R-squared:                       0.078
Model:                            OLS   Adj. R-squared:                  0.077
Method:                 Least Squares   F-statistic:                     142.7
Date:                Mon, 15 Mar 2021   Prob (F-statistic):               0.00
Time:                        00:26:05   Log-Likelihood:                -81677.
No. Observations:               23758   AIC:                         1.634e+05
Df Residuals:                   23743   BIC:                         1.635e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             7.1270      0.313     22.758      0.000       6.513       7.741
C(Bizgroup)[T.1]                      1.1206      0.310      3.614      0.000       0.513       1.728
C(post0313)[T.1]                     -2.6453      0.304     -8.705      0.000      -3.241      -2.050
C(Bizgroup_post)[T.1]              9.558e+11   1.19e+12      0.806      0.420   -1.37e+12    3.28e+12
C(city)[T.Cheyenne]                   2.5489      0.165     15.436      0.000       2.225       2.873
C(city)[T.Cody]                      -3.1053      0.213    -14.571      0.000      -3.523      -2.688
C(city)[T.Evanston]                  -0.8371      0.252     -3.317      0.001      -1.332      -0.342
C(city)[T.Gillette]                   0.5938      0.193      3.073      0.002       0.215       0.973
C(city)[T.Jackson]                   -2.3491      0.201    -11.684      0.000      -2.743      -1.955
C(city)[T.Laramie]                   -0.1319      0.199     -0.663      0.507      -0.522       0.258
C(city)[T.Rawlins]                    1.9023      0.243      7.817      0.000       1.425       2.379
C(city)[T.Rock Springs]              -0.0748      0.210     -0.356      0.722      -0.486       0.337
C(city)[T.Sheridan]                  -0.7720      0.215     -3.594      0.000      -1.193      -0.351
C(weekend)[T.1]                      -0.3426      0.113     -3.022      0.003      -0.565      -0.120
C(Bizgroup)[T.1]:C(post0313)[T.1] -9.558e+11   1.19e+12     -0.806      0.420   -3.28e+12    1.37e+12
==============================================================================
Omnibus:                    27031.807   Durbin-Watson:                   0.639
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          4350706.478
Skew:                           5.765   Prob(JB):                         0.00
Kurtosis:                      68.284   Cond. No.                     6.41e+13
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.02e-23. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

Answer to 4a: The estimated coefficient on Bizgroup is 1.1206. So if the business is dining services then daily visits is predicted to be 1.1206 higher in daily visits than a business that is hotel service. The coefficient on post0313 means if the date is March 13, ans business is dining services then daily visits is predicted to be 1.1206 lower in daily visits than a business that is hotel service. Bizgroup_post coefficient means if the business is dining services and date is March 13, then daily visits is predicted to be 9.56 higher in daily visits than a business that is hotel service. Weekend coefficient means if the business is dining services and it is a weekend then daily visits is predicted to be .3426 lower in daily visits than a business that is hotel service. Bizgroup is statiscally significant from zero. Bizgroup_post is not statiscally significant from zero. Post0313 is statiscally significant from zero. Weekend is statistically different. What is the economic meaning of each coefficient? I reject the Ho.

In [32]:
def log_plus_1(x):
    return np.log(x + 1)
results = smf.ols('log_plus_1(daily_visits) ~ C(Bizgroup) * C(post0313) + C(Bizgroup_post) + C(city) + C(weekend)', data=df).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                               
====================================================================================
Dep. Variable:     log_plus_1(daily_visits)   R-squared:                       0.133
Model:                                  OLS   Adj. R-squared:                  0.133
Method:                       Least Squares   F-statistic:                     261.3
Date:                      Mon, 15 Mar 2021   Prob (F-statistic):               0.00
Time:                              00:26:07   Log-Likelihood:                -30793.
No. Observations:                     23758   AIC:                         6.162e+04
Df Residuals:                         23743   BIC:                         6.174e+04
Df Model:                                14                                         
Covariance Type:                  nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             1.8214      0.037     49.522      0.000       1.749       1.894
C(Bizgroup)[T.1]                      0.0385      0.036      1.056      0.291      -0.033       0.110
C(post0313)[T.1]                     -0.4216      0.036    -11.814      0.000      -0.492      -0.352
C(Bizgroup_post)[T.1]               1.91e+11   1.39e+11      1.371      0.170    -8.2e+10    4.64e+11
C(city)[T.Cheyenne]                   0.2760      0.019     14.232      0.000       0.238       0.314
C(city)[T.Cody]                      -0.6389      0.025    -25.524      0.000      -0.688      -0.590
C(city)[T.Evanston]                  -0.1847      0.030     -6.231      0.000      -0.243      -0.127
C(city)[T.Gillette]                   0.1066      0.023      4.696      0.000       0.062       0.151
C(city)[T.Jackson]                   -0.5514      0.024    -23.352      0.000      -0.598      -0.505
C(city)[T.Laramie]                   -0.0971      0.023     -4.156      0.000      -0.143      -0.051
C(city)[T.Rawlins]                   -0.0080      0.029     -0.280      0.780      -0.064       0.048
C(city)[T.Rock Springs]              -0.0114      0.025     -0.462      0.644      -0.060       0.037
C(city)[T.Sheridan]                  -0.1141      0.025     -4.522      0.000      -0.164      -0.065
C(weekend)[T.1]                      -0.1317      0.013     -9.897      0.000      -0.158      -0.106
C(Bizgroup)[T.1]:C(post0313)[T.1]  -1.91e+11   1.39e+11     -1.371      0.170   -4.64e+11     8.2e+10
==============================================================================
Omnibus:                      274.040   Durbin-Watson:                   0.874
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              258.173
Skew:                           0.221   Prob(JB):                     8.68e-57
Kurtosis:                       2.745   Cond. No.                     6.41e+13
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.02e-23. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

# Question 4b: How would your answer to Question 4a change if (1) you add city fixed effects? (2) you add date fixed effects? and (3) you add both city fixed effects and date fixed effects? Which specification would you prefer for the hypothesis testing?

In [33]:
df2 = df.copy()
df2 = df2.set_index(['city', 'date'])
df2.head()

id  Rank date_range_start date_range_end  \
city   date                                                         
Casper 2020-03-09      2694     1       2020-03-09     2020-03-16   
       2020-03-16   3859526     1       2020-03-16     2020-03-23   
       2020-03-23   7601965     1       2020-03-23     2020-03-30   
       2020-03-30  11239563     1       2020-03-30     2020-04-06   
       2020-03-10      2694     2       2020-03-09     2020-03-16   

                     visits_by_day region  raw_visitor_counts  \
city   date                                                     
Casper 2020-03-09   5,8,5,4,14,8,5     WY                44.0   
       2020-03-16  10,3,6,5,10,0,1     WY                25.0   
       2020-03-23    6,3,1,4,5,2,0     WY                19.0   
       2020-03-30    0,0,2,4,7,1,1     WY                13.0   
       2020-03-10   5,8,5,4,14,8,5     WY                44.0   

                   raw_visit_counts                   safegraph_place_id  \
city   date                                                                
Casper 2020-03-09              49.0  sg:5380bbb5685d4e6eb8427f4b1d418772   
       2020-03-16              35.0  sg:5380bbb5685d4e6eb8427f4b1d418772   
       2020-03-23              21.0  sg:5380bbb5685d4e6eb8427f4b1d418772   
       2020-03-30              15.0  sg:5380bbb5685d4e6eb8427f4b1d418772   
       2020-03-10              49.0  sg:5380bbb5685d4e6eb8427f4b1d418772   

                   daily_visits dayname  positive_visits  weekend  \
city   date                                                         
Casper 2020-03-09           5.0     Mon                1        0   
       2020-03-16          10.0     Mon                1        0   
       2020-03-23           6.0     Mon                1        0   
       2020-03-30           0.0     Mon                0        0   
       2020-03-10           8.0     Tue                1        0   

                  location_name                         top_category  \
city   date                                                            
Casper 2020-03-09  Tacos Mexico  Restaurants and Other Eating Places   
       2020-03-16  Tacos Mexico  Restaurants and Other Eating Places   
       2020-03-23  Tacos Mexico  Restaurants and Other Eating Places   
       2020-03-30  Tacos Mexico  Restaurants and Other Eating Places   
       2020-03-10  Tacos Mexico  Restaurants and Other Eating Places   

                               sub_category naics_code  latitude   longitude  \
city   date                                                                    
Casper 2020-03-09  Full-Service Restaurants     722511  42.83767 -106.300181   
       2020-03-16  Full-Service Restaurants     722511  42.83767 -106.300181   
       2020-03-23  Full-Service Restaurants     722511  42.83767 -106.300181   
       2020-03-30  Full-Service Restaurants     722511  42.83767 -106.300181   
       2020-03-10  Full-Service Restaurants     722511  42.83767 -106.300181   

                  postal_code  Bizgroup  post0313  Bizgroup_post  
city   date                                                       
Casper 2020-03-09       82601         1         0              0  
       2020-03-16       82601         1         1              1  
       2020-03-23       82601         1         1              1  
       2020-03-30       82601         1         1              1  
       2020-03-10       82601         1         0              0

In [34]:
df2['BizgoupXpost0313'] = df2['Bizgroup'] * df2['post0313']

In [35]:
mod = PanelOLS(log_plus_1(df2['daily_visits']), df2[['Bizgroup', 'post0313', 'Bizgroup_post', 'weekend']],
               entity_effects = True,
               drop_absorbed = True)
results = mod.fit(cov_types = 'clustered')

print(results)

                          PanelOLS Estimation Summary                           
Dep. Variable:           daily_visits   R-squared:                        0.0556
Estimator:                   PanelOLS   R-squared (Between):             -0.6642
No. Observations:               23758   R-squared (Within):               0.0556
Date:                Mon, Mar 15 2021   R-squared (Overall):             -0.4237
Time:                        00:26:08   Log-likelihood                -3.079e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      349.28
Entities:                          10   P-value                           0.0000
Avg Obs:                       2375.8   Distribution:                 F(4,23744)
Min Obs:                       1169.0                                           
Max Obs:                       4606.0   F-statistic (robust):             349.28
                            

In [36]:
mod2 = PanelOLS(log_plus_1(df2['daily_visits']), df2[['Bizgroup', 'post0313', 'Bizgroup_post','weekend']],
               time_effects = True,
               drop_absorbed = True)
results2 = mod2.fit(cov_type='clustered')

print(results2)

C:\ProgramData\Anaconda3\lib\site-packages\linearmodels\panel\model.py:1743: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

post0313, weekend

  warnings.warn(


                          PanelOLS Estimation Summary                           
Dep. Variable:           daily_visits   R-squared:                        0.0012
Estimator:                   PanelOLS   R-squared (Between):              0.0546
No. Observations:               23758   R-squared (Within):               0.0154
Date:                Mon, Mar 15 2021   R-squared (Overall):              0.0424
Time:                        00:26:09   Log-likelihood                -3.151e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      14.106
Entities:                          10   P-value                           0.0000
Avg Obs:                       2375.8   Distribution:                 F(2,23728)
Min Obs:                       1169.0                                           
Max Obs:                       4606.0   F-statistic (robust):             13.857
                            

In [37]:
mod3 = PanelOLS(log_plus_1(df2['daily_visits']), df2[['Bizgroup', 'post0313']],
               entity_effects = True,
               time_effects = True,
               drop_absorbed = True)

results3 = mod3.fit(cov_type='clustered')

print(results3)

C:\ProgramData\Anaconda3\lib\site-packages\linearmodels\panel\model.py:1743: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

post0313

  warnings.warn(


                          PanelOLS Estimation Summary                           
Dep. Variable:           daily_visits   R-squared:                        0.0004
Estimator:                   PanelOLS   R-squared (Between):             -0.0415
No. Observations:               23758   R-squared (Within):               0.0003
Date:                Mon, Mar 15 2021   R-squared (Overall):             -0.0286
Time:                        00:26:10   Log-likelihood                -3.038e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      8.6557
Entities:                          10   P-value                           0.0033
Avg Obs:                       2375.8   Distribution:                 F(1,23720)
Min Obs:                       1169.0                                           
Max Obs:                       4606.0   F-statistic (robust):             9.1020
                            